In [1]:
// $$jcpp_file: src/main.cpp
// $$jcpp_build: mkdir -p bin && g++ -o3 -o bin/main src/main.cpp -fopenmp && TSP_FILE="./data/test.tsp" bin/main


#IFDEF JCPP_MAIN
#DEFINE JCPP_MAIN_METHOD
int main(){
		std::cout << "Hello World" << std::endl;
		return 0;
}


#ENDIF



// $$jcpp_file: src/main.cpp
// $$jcpp_build: mkdir -p bin && g++ -o3 -o bin/main src/main.cpp -fopenmp && TSP_FILE="./data/test.tsp" bin/main


#IFDEF JCPP_MAIN
#DEFINE JCPP_MAIN_METHOD
int main(){
		std::cout << "Hello World" << std::endl;
		return 0;
}


#ENDIF

 world vscode-notebook-cell:/lfiles/op2/main/test.ipynb#W0sZmlsZQ%3D%3D
src/main.cpp